In [35]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import gensim
from sklearn.base import BaseEstimator, TransformerMixin
import torch
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import pyLDAvis
import pyLDAvis.lda_model
import time
from gensim.models import LdaModel, CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from gensim.utils import simple_preprocess
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier

[nltk_data] Downloading package stopwords to /Users/bahia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Metriques d'évaluation utilisées
  
  
Perplexity : La perplexité est une mesure de la qualité du modèle. Une perplexité plus faible indique un meilleur modèle. Vous pouvez obtenir ce score en utilisant la méthode perplexity.  
  
Coherence Score : Le score de cohérence mesure la cohérence sémantique des sujets générés par le modèle LDA. Un score de cohérence plus élevé indique que les mots les plus importants d'un sujet sont plus sémantiquement similaires les uns aux autres.  
  
### LDA.  
L'Analyse de la Répartition Latente de Dirichlet (LDA) est une technique de modélisation de sujets qui est largement utilisée en traitement du langage naturel. Elle permet de découvrir les sujets cachés dans un ensemble de documents en attribuant des distributions de probabilité à la fois aux documents (sur les sujets) et aux sujets (sur les mots), ce qui permet d'obtenir une représentation non supervisée et interprétable des données textuelles.



### Lda avec gensim

In [36]:

# Classe de prétraitement
class GensimPreprocessor:
    def __init__(self, stop_words=stopwords.words('english')):
        self.stop_words = stop_words
        
    def tokenize(self, text):
        return [word for word in word_tokenize(text.lower()) if word.isalpha()]
    
    def remove_stopwords(self, tokens):
        return [word for word in tokens if word not in self.stop_words]
    
    def preprocess(self, documents):
        preprocessed = []
        for document in documents:
            tokens = self.tokenize(document)
            tokens = self.remove_stopwords(tokens)
            preprocessed.append(tokens)
        return preprocessed

# Classe pour l'application de LDA de Gensim
'''class GensimLDA:
    def __init__(self, num_topics=8, passes=10):
        self.num_topics = num_topics
        self.passes = passes
        self.dictionary = None
        self.model = None
        
    def fit(self, documents):
        self.dictionary = Dictionary(documents)
        corpus = [self.dictionary.doc2bow(text) for text in documents]
        self.model = LdaModel(corpus=corpus, id2word=self.dictionary, num_topics=self.num_topics, passes=self.passes, random_state=42)
        
    def get_coherence(self, documents):
        coherence_model = CoherenceModel(model=self.model, texts=documents, dictionary=self.dictionary, coherence='c_v')
        return coherence_model.get_coherence()
'''

"class GensimLDA:\n    def __init__(self, num_topics=8, passes=10):\n        self.num_topics = num_topics\n        self.passes = passes\n        self.dictionary = None\n        self.model = None\n        \n    def fit(self, documents):\n        self.dictionary = Dictionary(documents)\n        corpus = [self.dictionary.doc2bow(text) for text in documents]\n        self.model = LdaModel(corpus=corpus, id2word=self.dictionary, num_topics=self.num_topics, passes=self.passes, random_state=42)\n        \n    def get_coherence(self, documents):\n        coherence_model = CoherenceModel(model=self.model, texts=documents, dictionary=self.dictionary, coherence='c_v')\n        return coherence_model.get_coherence()\n"

### Functions

In [37]:
def words_tokenize(text):
  text = text.split()
  return text

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    time_list = []
    for num_topics in range(start, limit, step):
        start_time = time.time()
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        end_time = time.time()
        time_list.append(end_time - start_time)

    return model_list, coherence_values, time_list

In [38]:
data = pd.read_csv('Data/df_tags_filtered.csv')
data['combined'] = data['Title'] + " " + data['Body']
data['Tags'] = data.Tags.apply(lambda text: words_tokenize(text))
display(data[:1])

,Title,Body,Tags,Body_word_count,combined
0,android jetpack navigation bottomnavigationvie...,android jetpack navigation bottomnavigationvie...,"[android, android-architecture-components, bot...",131,android jetpack navigation bottomnavigationvie...


In [39]:
X = data['combined'][:100]
y = data['Tags'][:100]

#df = pd.DataFrame(data)

'''X_train, X_test, y_train, y_test = train_test_split(X,y,
    test_size=0.2,
    random_state=42
)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)'''


'X_train, X_test, y_train, y_test = train_test_split(X,y,\n    test_size=0.2,\n    random_state=42\n)\n\nprint("X_train shape:", X_train.shape)\nprint("X_test shape:", X_test.shape)\nprint("y_train shape:", y_train.shape)\nprint("y_test shape:", y_test.shape)'

### nombre optimal de topics

In [40]:
#documents = X_train
documents = X

# Preprocess the text data
preprocessor = GensimPreprocessor()
texts = preprocessor.preprocess(documents)

# Create the Dictionary
dictionary = Dictionary(texts)

# Create the Corpus
corpus = [dictionary.doc2bow(text) for text in texts]

# Call the function and plot the results
'''model_list, coherence_values, time_list = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=texts, start=5, limit=15, step=1)

x = range(5, 15, 1)
plt.figure(figsize=(20, 8))
plt.plot(x, coherence_values, label='Coherence')
plt.xlabel("Num Topics")
plt.legend(loc='best')
plt.show()'''

'model_list, coherence_values, time_list = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=texts, start=5, limit=15, step=1)\n\nx = range(5, 15, 1)\nplt.figure(figsize=(20, 8))\nplt.plot(x, coherence_values, label=\'Coherence\')\nplt.xlabel("Num Topics")\nplt.legend(loc=\'best\')\nplt.show()'

In [41]:
# Split your documents into a training and test set
texts = preprocessor.preprocess(X)
#dictionnary = Dictionary(texts_for_test)
'''train_texts = texts_for_test[:int(len(texts_for_test)*0.8)]
test_texts = texts_for_test[int(len(texts_for_test)*0.8):]'''

# Create a dictionary and corpus from the training set
#train_dictionary = dictionary
#train_corpus = [train_dictionary.doc2bow(text) for text in train_texts]

#test_corpus = [train_dictionary.doc2bow(text) for text in test_texts]

# Train LDA model on the training set
lda = LdaModel(corpus=corpus, id2word=dictionary, topics=8, passes=10)

# Evaluate model on the test set
# For perplexity
log_perplexity = lda.log_perplexity(corpus)
perplexity = np.exp(-log_perplexity)
print(f"Perplexity on train set: {perplexity}")

log_perplexity = lda.log_perplexity(corpus)
perplexity = np.exp(-log_perplexity)
print(f"Perplexity on test set: {perplexity}")

# For coherence
coherence_model_lda_train = CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_score_test = coherence_model_lda_train.get_coherence()
print("Coherence score on train set =", coherence_score_test)

coherence_model_lda_test = CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_score_test = coherence_model_lda_test.get_coherence()
print("Coherence score on test set =", coherence_score_test)


TypeError: __init__() got an unexpected keyword argument 'topics'

In [ ]:
import pyLDAvis.gensim_models
import pyLDAvis
# Assurez-vous d'appeler pyLDAvis.enable_notebook() dans un environnement Jupyter pour l'interaction inline
pyLDAvis.enable_notebook()

# Préparation des données pour PyLDAVis
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary=dictionary)

# Visualisation
pyLDAvis.display(vis)

In [ ]:
# obtain topic distributions for each document
test_documents_topics = lda_train.get_document_topics(test_corpus, minimum_probability=0)
topic_dist = pd.DataFrame(test_documents_topics)
for topic in topic_dist.columns:
    topic_dist[topic] = topic_dist[topic].apply(lambda x : x[1])

print('matrix of document/topic distribution shape:', topic_dist.shape)
display(topic_dist.head())


matrix of document/topic distribution shape: (13, 13)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.002833,0.203966,0.120959,0.045232,0.002833,0.002833,0.002833,0.002833,0.083529,0.002833,0.002833,0.275515,0.250968
1,0.001336,0.153470,0.142222,0.001336,0.033142,0.001336,0.001336,0.001336,0.135417,0.072479,0.112850,0.342401,0.001336
2,0.001341,0.082710,0.001341,0.108236,0.055005,0.001341,0.190470,0.043893,0.175574,0.051099,0.205079,0.001341,0.082572
3,0.000754,0.000754,0.149508,0.000754,0.000754,0.000754,0.229824,0.106624,0.111242,0.000754,0.146527,0.168288,0.083465
4,0.001277,0.706566,0.001277,0.001277,0.001277,0.048079,0.088740,0.001277,0.017750,0.001277,0.128650,0.001277,0.001277


### regression logistique

In [ ]:
mlb = MultiLabelBinarizer()
y_test_binarized = mlb.fit_transform(test_texts)
time1 =time.time()
x_train, x_test, y_train_lr, y_test_lr = train_test_split(topic_dist, y_test_binarized, test_size=0.2, random_state=100)
lr = OneVsRestClassifier(LogisticRegression()).fit(x_train, y_train_lr)
y_pred_lr = lr.predict_proba(x_test)
fit_time = np.round(time.time() - time1,1)
#score of prediction



/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/multiclass.py:84: UserWarning: Label not 5 is present in all training examples.
  warnings.warn(
/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/multiclass.py:84: UserWarning: Label not 26 is present in all training examples.
  warnings.warn(
/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/multiclass.py:84: UserWarning: Label not 28 is present in all training examples.
  warnings.warn(
/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/multiclass.py:84: UserWarning: Label not 29 is present in all training examples.
  warnings.warn(
/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/multiclass.py:84: UserWarning: Label not 33 is present in all training examples.
  warnings.warn(
/Users/bahia/opt/anaconda3/lib/python3.9/site-packages/sklearn/multiclass.py:84: UserWarning: Label not 40 is present in all training examples.
  warnings.warn(
/Users/bahia/opt/anaconda3/lib/pyth

In [ ]:
probabilities = y_pred_lr

# Définir un seuil personnalisé pour sélectionner les tags
threshold = 0.15  # baisser le seuil pour inclure plus de tags

def select_top_n_tags(probabilities, threshold, top_n=5):
    # Créer une matrice de zéros de la même forme que probabilities
    final_tags = np.zeros(probabilities.shape)
    
    for i in range(probabilities.shape[0]):
        # Trouver les indices qui satisfont le seuil
        eligible_indices = np.where(probabilities[i] >= threshold)[0]
        
        # Trier ces indices en fonction de leur probabilité
        top_indices = eligible_indices[np.argsort(probabilities[i][eligible_indices])][::-1]
        
        # Sélectionner les top N indices
        top_n_indices = top_indices[:top_n]
        
        # Mettre à 1 les positions correspondant aux tags sélectionnés
        final_tags[i, top_n_indices] = 1
    
    return final_tags

# Utiliser la fonction pour obtenir les tags prédits

binary_predictions = select_top_n_tags(probabilities, threshold, top_n=7)

In [ ]:
from sklearn.metrics import jaccard_score


print("jaccord score:", jaccard_score(binary_predictions, y_test_lr, average='samples'))
print("Precision:", precision_score(binary_predictions, y_test_lr, average='samples'))
print("Recall:", recall_score(binary_predictions, y_test_lr, average='samples'))
print("F1 Score:", f1_score(binary_predictions, y_test_lr, average='samples'))
'''jaccord score: 0.16363636363636364
Precision: 0.3333333333333333
Recall: 0.23809523809523805
F1 Score: 0.27777777777777773'''


jaccord score: 0.028619528619528618
Precision: 0.032345679012345675
Recall: 0.19047619047619047
F1 Score: 0.05469556243550052


'jaccord score: 0.16363636363636364\nPrecision: 0.3333333333333333\nRecall: 0.23809523809523805\nF1 Score: 0.27777777777777773'

In [ ]:
# Afficher les labels des prédictions
predictions_labels = mlb.inverse_transform(binary_predictions)
true_labels = mlb.inverse_transform(y_test_lr)
print(predictions_labels)
print(true_labels)

[('android', 'call', 'code', 'fragment', 'issue', 'override', 'work'), ('android', 'call', 'code', 'fragment', 'issue', 'override', 'work'), ('android', 'call', 'code', 'fragment', 'issue', 'override', 'work')]
[('access', 'activity', 'android', 'androidauthoritiescomxxxxwordscontentprovidermaincontentprovider', 'androidnamecontentprovidermaincontentprovidergt', 'argument', 'bundle', 'call', 'certain', 'comxxxxwordscontentprovider', 'content', 'contentcomxxxxwordscontentproviderwords', 'contentresolver', 'contenturi', 'contentvalues', 'detailsactivity', 'different', 'display', 'error', 'extends', 'file', 'fragment', 'getcontentresolver', 'getcontentresolverinserturi', 'id', 'implement', 'initial', 'intent', 'item', 'javalangillegalargumentexception', 'list', 'listfragment', 'loadermanagerloadercallbacksltcursorgt', 'ltprovider', 'mainactivity', 'mainfragment', 'manifest', 'method', 'provider', 'resolver', 'return', 'right', 'situation', 'unknown', 'uri', 'uriparsecontent', 'uriparsemai

In [ ]:


# Créer un DataFrame à partir des labels de prédiction et des vrais labels
df = pd.DataFrame({
    'Predicted Labels': predictions_labels,
    'True Labels': true_labels
})

display(df)

,Predicted Labels,True Labels
0,"(android, call, code, fragment, issue, overrid...","(access, activity, android, androidauthorities..."
1,"(android, call, code, fragment, issue, overrid...","(androidhintdescrio, androidididettextodescric..."
2,"(android, call, code, fragment, issue, overrid...","(achieve, androidbackgroundcolorblue, androidb..."
